In [1]:
!pip install depthai

In [4]:
!pip install opencv-python

In [7]:
import cv2 as cv
import depthai as dai
import numpy as np
import time

#### 4

In [9]:
# Closer-in minimum depth, disparity range is doubled (from 95 to 190):
extended_disp = False
# Better accuracy for longer distance, fractional disparity 32-levels:
sub_pixelel = False
# Better handling for occlusions:
lr_check = True

# Create pipeline
pipeline = dai.Pipeline()

# camera configurations
camera_rgb = pipeline.create(dai.node.ColorCamera)
camera_rgb.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)
xout_rgb = pipeline.createXLinkOut()
xout_rgb.setStreamName("rgb")
camera_rgb.video.link(xout_rgb.input)
left = pipeline.createMonoCamera()
left.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
left.setBoardSocket(dai.CameraBoardSocket.LEFT)
right = pipeline.createMonoCamera()
right.setResolution(dai.MonoCameraProperties.SensorResolution.THE_400_P)
right.setBoardSocket(dai.CameraBoardSocket.RIGHT)
depth = pipeline.createStereoDepth()
left.out.link(depth.left)
right.out.link(depth.right)
xout = pipeline.createXLinkOut()
xout.setStreamName("disparity")
depth.disparity.link(xout.input)

prev_frame_time = 0
new_frame_time = 0
DIM = (720, 480)

C:\Users\Owner\AppData\Local\Temp\ipykernel_21604\1247709747.py:19: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  left.setBoardSocket(dai.CameraBoardSocket.LEFT)
C:\Users\Owner\AppData\Local\Temp\ipykernel_21604\1247709747.py:22: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  right.setBoardSocket(dai.CameraBoardSocket.RIGHT)


In [10]:
with dai.Device(pipeline) as device:

    # Output queue will be used to get the disparity frames from the outputs defined above
    q = device.getOutputQueue(name="disparity", maxSize=4, blocking=False)
    q_rgb = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)
    while True:

        new_frame_time = time.time()

        fps = 1 / (new_frame_time - prev_frame_time)
        prev_frame_time = new_frame_time
        fps = int(fps)

        in_rgb = q_rgb.get()

        in_disparity = q.get()  # blocking call, will wait until a new data has arrived
        frame = in_disparity.getFrame()

        # Normalization for better visualization
        frame = (frame * (255 / depth.initialConfig.getMaxDisparity())).astype(np.uint8)

        frame_rgb = cv.resize(in_rgb.getCvFrame(), DIM, interpolation=cv.INTER_AREA)

        cv.imshow("rgb", frame_rgb)
        cv.imshow("disparity", frame)

        print("FPS: ", fps)
        if cv.waitKey(1) == ord('q'):
            break

RuntimeError: Device already closed or disconnected: io error

In [11]:
import numpy as np
import cv2 as cv
import glob
import time
from pathlib import Path
import depthai as dai

##### 1, 2.

In [13]:
'''
    function to capture 10 images from mentioned source 
    - source can be RIGHT or LEFT monochrome camera of OAK D LITE
    - images captured at 1000 sec interval
    
    params : 
    
        src = {'right' || 'left'} (default : right)
        delay = {delay in ms} (default : 1000)
'''

def captureImages(src='right', delay=1000):
    if src != 'right' and src != 'left': 
        print("ENTER CORRECT PARAMS!")
        print("accepted params: {left, right} ")
        print(f"entered src:{src}")
        return;
    
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - mono (grayscale) camera
    # LEFT or RIGHT    
    
    cam = pipeline.createMonoCamera()
    
    if src == 'right' :
        cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
    else :
        cam.setBoardSocket(dai.CameraBoardSocket.LEFT)

    cam.setResolution(dai.MonoCameraProperties.SensorResolution.THE_480_P)

    # Create output
    xout = pipeline.createXLinkOut()
    xout.setStreamName(src)
    cam.out.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline,usb2Mode=True) as device:

        # Output queue will be used to get the grayscale frames from the output defined above
        q = device.getOutputQueue(name=src, maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/{src}").mkdir(parents=True, exist_ok=True)

        for i in range(10):
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            cv.imshow(src, frame)

            cv.imwrite(f"images/{src}/{int(time.time() * 10000)}.png", frame)
            cv.waitKey(delay)  

            cv.destroyAllWindows()            


def captureColorImages(delay=1000):
    
    # Start defining a pipeline
    pipeline = dai.Pipeline()

    # Define a source - color camera
    
    cam = pipeline.createColorCamera()
    cam.setResolution(dai.ColorCameraProperties.SensorResolution.THE_1080_P)

    # Create RGB output
    xout = pipeline.createXLinkOut()
    xout.setStreamName("rgb")
    cam.video.link(xout.input)

    # Connect and start the pipeline
    with dai.Device(pipeline,usb2Mode=True) as device:

        # Output queue will be used to get the color frames from the output defined above
        q = device.getOutputQueue(name="rgb", maxSize=4, blocking=False)

        # Make sure the destination path is present before starting to store the examples
        Path(f"images/rgb").mkdir(parents=True, exist_ok=True)

        for i in range(10):
            # Blocking call, will wait until a new data has arrived
            inSrc = q.get()  
            # Data is originally represented as a flat 1D array, it needs to be converted into HxW form
            frame = inSrc.getCvFrame()
            # Frame is transformed and ready to be shown
            imS = cv.resize(frame, (960, 540)) # Resize image
            cv.imshow("rgb", imS)   
#             cv.imshow("rgb", frame)

            cv.imwrite(f"images/rgb/{int(time.time() * 10000)}.png", frame)
            cv.waitKey(delay)  

            cv.destroyAllWindows()

In [1]:
'''
    function to find corners, caliberate and store 
    camera matrix and distortion vector from mentioned source 
    - source can be RIGHT or LEFT monochrome camera or COLOR camera 
    of OAK D LITE
    
    params : 
        images = {array of image paths}
        src = source file {'right' || 'left' || 'rgb'} 
'''

def caliberate(images, src):
    if src != 'right' and src != 'left' and src != 'rgb': 
        print("ENTER CORRECT PARAMS!")
        print("accepted params: {left, right, rgb} ")
        print(f"entered src:{src}")
        return;

    
    # termination criteria
    criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

    # Arrays to store object points and image points from all the images.
    objpoints = [] # 3d point in real world space
    imgpoints = [] # 2d points in image plane.

    notFound = []

#     img_size = () # will be using this for caliberation

    for fname in images:
        img = cv.imread(fname)
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
        
#         img_size = gray.shape[::-1]
    
        cv.imshow('gray', img)        
        cv.waitKey(1000)
        cv.destroyAllWindows()

        # Finding chess board corners
        ret, corners = cv.findChessboardCorners(gray, (9,6), None)

        # If found, add object points, image points (after refining them)
        if ret == True:
            objpoints.append(objp)
            corners2 = cv.cornerSubPix(gray,corners, (11,11), (-1,-1), criteria)
            imgpoints.append(corners)

            # Draw and display the corners
            cv.drawChessboardCorners(img, (9,6),corners2, ret)
            cv.imshow('img', img)

            # Saving diplayed corners for future references
            cv.imwrite(f"{fname.split('.')[0]}_corners.png", img)
            cv.waitKey(1000)
            cv.destroyAllWindows()
        else :
            # if corners not found, storing it into a list

            notFound.append(fname)
            print(f"corners not found for {fname}")

    cv.destroyAllWindows()

    # removing the pictures whose corners werent found
    # from the main image list
    for i in notFound:
        images.remove(i)

    # calibration
    ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
        
    for fname in images:
        img = cv.imread(fname)
        h,  w = img.shape[:2]
        newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

        # undistort
        dst = cv.undistort(img, mtx, dist, None, newcameramtx)

        # crop the image
        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]
        
        # save new image into file
        cv.imwrite(f"{fname.split('.')[0]}_result.png", dst)
        
        
    # we will be storing the camera matrix and 
    # distortion coefficients for future uses

    print("Saving camera matrix...")
    camera_matrix = np.matrix(mtx)
    with open(f"images/{src}/camera_matrix.txt",'wb') as f:
        for line in camera_matrix:
            np.savetxt(f, line, fmt='%.5f')
            
    print("Saving distortion vector...")
    distortion_vector = np.matrix(dist)
    with open(f"images/{src}/distortion_matrix.txt",'wb') as f:
        for line in distortion_vector:
            np.savetxt(f, line, fmt='%.5f')
            
    
    print("Saving rotational vectors ...")
    rotation_vectors = np.array(rvecs)
    with open(f"images/{src}/rotat_vector.txt",'wb') as f:
        for vector in rotation_vectors:
            vector = np.reshape(vector, (1,3))
            np.savetxt(f, vector, fmt='%.5f')
            
    print("Saving translation vectors...")
    translation_vectors = np.array(tvecs)
    with open(f"images/{src}/trans_vector.txt",'wb') as f:
        for vector in translation_vectors:
            vector = np.reshape(vector, (1,3))
            np.savetxt(f, vector, fmt='%.5f')
            
    print('Done!!')

In [17]:
'''
    calling function to capture images
    
    I have used the previously declared function to 
    capture 10 images of a chessboard 
    
    these images are then used to caliberate the camera
    i used a 8x6 chess board for caliberations purposes
    
'''

# capturing images using right monochrome camera
captureImages('right')

# capturing images using left monochrome camera
captureImages('left')

# capturing images using color camera
captureColorImages()
right_images = glob.glob('images/right/*.png')
color_images = glob.glob('images/rgb/*.png')
left_images = glob.glob('images/left/*.png')

# bruh_images = glob.glob('images/bruh/*.png')

caliberate(right_images, 'right')
caliberate(color_images, 'rgb')
caliberate(left_images, 'left') 

C:\Users\Owner\AppData\Local\Temp\ipykernel_21604\3467920802.py:28: DeprecationWarning: RIGHT is deprecated, use CAM_C or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.RIGHT)
C:\Users\Owner\AppData\Local\Temp\ipykernel_21604\3467920802.py:40: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline,usb2Mode=True) as device:
C:\Users\Owner\AppData\Local\Temp\ipykernel_21604\3467920802.py:30: DeprecationWarning: LEFT is deprecated, use CAM_B or address camera by name  instead.
  cam.setBoardSocket(dai.CameraBoardSocket.LEFT)
C:\Users\Owner\AppData\Local\Temp\ipykernel_21604\3467920802.py:78: DeprecationWarning: Use constructor taking 'UsbSpeed' instead
  with dai.Device(pipeline,usb2Mode=True) as device:


corners not found for images/right\17132232119207.png
corners not found for images/right\17132232129884.png
corners not found for images/right\17132232151002.png
corners not found for images/right\17132232193151.png
corners not found for images/right\17132232203740.png
corners not found for images/right\17132232214171.png
Saving camera matrix...
Saving distortion vector...
Saving rotational vectors ...
Saving translation vectors...
Done!!
corners not found for images/rgb\17132232434939.png
corners not found for images/rgb\17132232445946.png
corners not found for images/rgb\17132232457044.png
corners not found for images/rgb\17132232479436.png
corners not found for images/rgb\17132232490395.png
corners not found for images/rgb\17132232501700.png
corners not found for images/rgb\17132232523973.png
corners not found for images/rgb\17132232534950.png
Saving camera matrix...
Saving distortion vector...
Saving rotational vectors ...
Saving translation vectors...
Done!!
corners not found for 

In [ ]:
right_images = glob.glob('images/right/*.png')
color_images = glob.glob('images/rgb/*.png')
left_images = glob.glob('images/left/*.png')

# bruh_images = glob.glob('images/bruh/*.png')

caliberate(right_images, 'right')
caliberate(color_images, 'rgb')
caliberate(left_images, 'left')

#### 3.

In [2]:
import numpy as np
import cv2 as cv
import glob
import time
from pathlib import Path
import depthai as dai
import math

In [3]:
camera_matrix = []
 
with open('images/right/camera_matrix.txt', 'r') as f:
    for line in f :
        camera_matrix.append([float(num) for num in line.split(' ')])

print("CAMERA INTRINSIC MATRIX:")
print(camera_matrix)

ValueError: could not convert string to float: ''

In [4]:
FX = 525
FY = 525
Z = 320

In [5]:
def convert_milli_to_inch(x):
    x = x / 10
    return x / 25.4
image = cv.imread("cv object image.jpeg")

In [6]:
x, y, w, h =15,16,18,1

In [7]:
cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 5)
Image_point1x = x
Image_point1y = y
Image_point2x = x + w
Image_point2y = y + h
cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 5)
cv.line(image, (Image_point1x, Image_point1y), (Image_point1x, Image_point2y), (0, 0, 255), 8)
Real_point1x = Z * (Image_point1x / FX)
Real_point1y = Z * (Image_point1y / FY)
Real_point2x = Z * (Image_point2x / FX)
Real_point2y = Z * (Image_point2x / FY)
print(Real_point1x)
print(Real_point1y)
print(Real_point2x)
print(Real_point2y)
dist = math.sqrt((Real_point2y - Real_point1y) ** 2 + (Real_point2x - Real_point1x) ** 2)
val = round(convert_milli_to_inch(dist*2), 5)
print("Diameter of blue cirlce: {}".format(val*100))
cv.putText(image, str(val) + " inches", (Image_point1x-200, (y + y + h) // 2 + 5),cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

9.142857142857142
9.752380952380953
20.114285714285714
20.114285714285714
Diameter of blue cirlce: 11.883000000000001


array([[[ 41,  60,  68],
        [ 46,  65,  73],
        [ 48,  67,  75],
        ...,
        [123, 137, 149],
        [123, 137, 149],
        [131, 145, 157]],

       [[ 25,  44,  52],
        [ 31,  50,  58],
        [ 34,  53,  61],
        ...,
        [153, 167, 179],
        [124, 138, 150],
        [108, 122, 134]],

       [[ 53,  70,  79],
        [ 53,  70,  79],
        [ 47,  64,  73],
        ...,
        [164, 178, 190],
        [164, 178, 190],
        [179, 193, 205]],

       ...,

       [[ 65,  77,  89],
        [ 63,  75,  87],
        [ 61,  73,  85],
        ...,
        [104, 119, 128],
        [ 99, 114, 123],
        [101, 116, 125]],

       [[ 44,  56,  68],
        [ 42,  54,  66],
        [ 40,  52,  64],
        ...,
        [139, 154, 163],
        [141, 156, 165],
        [136, 151, 160]],

       [[ 21,  33,  45],
        [ 20,  32,  44],
        [ 21,  33,  45],
        ...,
        [238, 253, 255],
        [234, 249, 255],
        [212, 227, 236]]

In [8]:
import cv2 as cv
import math

def get_circle_diameter(image_path, camera_matrix_path):
    def convert_milli_to_cm(x):
        x = x / 10
        return x / 25.4

    # Load camera matrix
    camera_matrix = []
    with open(camera_matrix_path, 'r') as f:
        for line in f:
            camera_matrix.append([float(num) for num in line.split(' ')])

    # Load image
    image = cv.imread(image_path)

    # Define points (you may need to adjust these values)
    x, y, w, h = 15, 16, 13, 1
    Image_point1x = x
    Image_point1y = y
    Image_point2x = x + w
    Image_point2y = y + h

    # Draw rectangle and line on the image
    cv.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 5)
    cv.line(image, (Image_point1x, Image_point1y), (Image_point1x, Image_point2y), (0, 0, 255), 8)

    # Calculate real world points
    Z = 310
    FX = camera_matrix[0][0]
    FY = camera_matrix[1][1]
    Real_point1x = Z * (Image_point1x / FX)
    Real_point1y = Z * (Image_point1y / FY)
    Real_point2x = Z * (Image_point2x / FX)
    Real_point2y = Z * (Image_point2y / FY)

    # Calculate diameter in pixels
    dist = math.sqrt((Real_point2y - Real_point1y) ** 2 + (Real_point2x - Real_point1x) ** 2)

    return dist

# Example usage:
image_path = "cv_object_image.jpeg"
camera_matrix_path = "images/right/camera_matrix.txt"
diameter_cm = get_circle_diameter(image_path, camera_matrix_path)
print("Diameter of blue circle: {} cm".format(diameter_cm))


ValueError: could not convert string to float: ''